<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [3]:
#!pip install imbalanced-learn -q
#!pip install swifter - q

#!pip install lightgbm
#!pip install xgboost

In [2]:
import pandas as pd
from tqdm import notebook
import re
from time import time 
#import nltk
import numpy as np
from numpy.random import default_rng
import transformers
from sklearn.model_selection import train_test_split,cross_val_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

from __future__ import unicode_literals, print_function
from spacy.lang.en import English

In [3]:
from imblearn.under_sampling import RandomUnderSampler

In [4]:
import torch
import transformers

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
import swifter

In [7]:
#import spacy
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#from spacy.pipeline import Lemmatizer
#!python -m spacy download en_core_web_sm
#nlp = spacy.load("en_core_web_sm")

In [8]:
#tokenizer = transformers.BertTokenizer(
#    vocab_file='datasets/ds_bert/vocab.txt')

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
nlp = English()
nlp.add_pipe('sentencizer')

In [11]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [12]:
tokenizer.encode('He ran. He fell.')

[101, 2002, 2743, 1012, 2002, 3062, 1012, 102]

In [13]:
tokenizer.added_tokens_encoder

{'[PAD]': 0, '[UNK]': 100, '[CLS]': 101, '[SEP]': 102, '[MASK]': 103}

In [14]:
for enc in tokenizer.encode('He ran. He fell. Aaaaaaaaaaa...'):
    display((enc,tokenizer.decode([f'{enc}'])))

(101, '[CLS]')

(2002, 'he')

(2743, 'ran')

(1012, '.')

(2002, 'he')

...

(11057, '##aa')

(1012, '.')

(1012, '.')

(1012, '.')

(102, '[SEP]')

In [15]:
RANDOM_STATE = 88

## Подготовка

In [16]:
start = time()
#df['text'].apply(lambda x: len(str(x).split()))
end = time()
end-start

0.0

In [17]:
url = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

In [18]:
df = pd.read_csv(url)

In [19]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [21]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [22]:
df['text'].duplicated().sum()

0

In [1510]:
def clean_text(string):
    string = re.sub(r'[^a-z]', ' ', string, flags=re.IGNORECASE)
    string = string.split()
    string = ' '.join(string)
    return string.lower()

In [1513]:
df['clean_text'] = df['text'].apply(lambda x:clean_text(x))

In [1514]:
df['clean_text'].duplicated().sum()

1323

In [1515]:
df.drop_duplicates(subset='clean_text', inplace=True)

In [1516]:
df.reset_index(drop=True, inplace=True)

In [23]:
df['text'].str.lower().duplicated().sum()

46

In [24]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [25]:
def sents(string):
    doc = nlp(string)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

In [26]:
df['word_count'] = df['text'].apply(lambda x: x.count(' ') + 1)

In [27]:
#df['sent_count'] = df['text'].apply(lambda x: len(sents(x)))

In [28]:
df['len'] = df['text'].apply(lambda x: len(x))

In [29]:
df['text'][154485]

'"\n\n ""Disappointing Players for Charlton"" \n\nI added in a section on the April Fools prank in the Crest\'s section, as well as splitting the directors of the boardroom into two sections, Charlton Athletic Ltd & CAFC PLC.\n\n"'

In [30]:
#df['text'][112719]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        159292 non-null  object
 1   toxic       159292 non-null  int64 
 2   word_count  159292 non-null  int64 
 3   len         159292 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.9+ MB


In [32]:
#df['text_lower'] = df['mid_text'].str.lower()
#df.drop_duplicates(subset='text_lower', inplace=True)
#df.reset_index(drop=True, inplace=True)

In [33]:
def distribution_plt(df_list, name, val_list, plt_xlim=1):

    x_max = min([series.max() for series in df_list])* plt_xlim

    fig, ax = plt.subplots(2,1,figsize=(10,8))
    ax[0].boxplot([df for df in df_list], vert=False)

    ax[0].set_yticklabels(val_list)
    ax[0].set_xlabel(name)
    ax[0].set_xlim([0, x_max])

    ax[1].hist([df for df in df_list], histtype='stepfilled', bins=500)
    ax[1].set_ylabel('частота')
    ax[1].set_xlabel(name)
    ax[1].set_xlim([0, x_max])

    
    plt.suptitle(f'Ящики с усами и гистограмма для признака "{name}"')
    plt.legend(val_list);
    #plt.xlim((0,x_max))

In [34]:
name_dict = {'word_count' : 'количество слов',
 'sent_count' : 'количество предложений',
 'len' : 'количество символов'}

for col in ['word_count', 'sent_count', 'len']:
    
    distribution_plt(
        
        [df[df['toxic']==1][col], df[df['toxic']==0][col]],
        
        name_dict[col],
        
        ['токсичные\nкомментарии', 'не токсичные\nкомментарии'],
        
        0.1)

In [35]:
def word_rebuild(word_list):
    
    for word in range(len(word_list)):
        if len(word_list[word])>100:
            word_list[word] = ''.join(list(dict.fromkeys(word_list[word])))
    
    return word_list

In [42]:
config = transformers.BertConfig.from_dict({
      "attention_probs_dropout_prob": 0.1,
      "directionality": "bidi",
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.1,
      "hidden_size": 768,
      "initializer_range": 0.02,
      "intermediate_size": 3072,
      #"layer_norm_eps": 7e-2,
      #"layer_norm_eps": 1e-12,
      "layer_norm_eps": 4e-5,
      "max_position_embeddings": 512,
      "model_type": "bert",
      "num_attention_heads": 12,
      "num_hidden_layers": 12,
      "pad_token_id": 0,
      "pooler_fc_size": 768,
      "pooler_num_attention_heads": 12,
      "pooler_num_fc_layers": 3,
      "pooler_size_per_head": 128,
      "pooler_type": "first_token_transform",
      #"position_embedding_type": "absolute",
      #"transformers_version": "4.12.5",
      "use_cache":True,
      "type_vocab_size": 2,
      "vocab_size": 30522
}
)

In [646]:
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.50.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

def chunker(string, max_tokens):

    sents_ = sents(string)

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    
    if len(toks)>1:
        result = sum(toks,[])    
    else:
        result = toks[0]
    
    if len(result)>max_tokens:
        
        mid_sent = len(toks)//2
        
        if mid_sent == 1:
            
            if len(toks[1])<max_tokens:
                result = toks[1]    
            else:
                result = toks[0]
                
        elif mid_sent > 1:
            
            for s in range(1,mid_sent):

                mid_sents = toks[(mid_sent-s):mid_sent] + toks[mid_sent:(mid_sent+s)]#+toks[-1]
                mid_sents_s = sum(mid_sents,[])
                if len(mid_sents_s) > max_tokens:
                    break
                    result = sum(mid_sents[:-1],[])
        
        else:
            result = sum(toks,[])
    #return result
    return result[:max_tokens][:-1]+[102]

def chunker(string, max_tokens):

    max_tokens = max_tokens-1
    sents_ = sents(string)

    
    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=False) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    

    result = sum(toks,[])    
    
    if len(result)>max_tokens:
        if len(sents_)>2:
            result = toks[0]+toks[-1]
    
            
    return [101] + result[:max_tokens-2] + [102]

In [44]:
def chunker(string, max_tokens=512):
    
    tokenized = tokenizer.encode(string, add_special_tokens=True)
    len_wo_tokens = max_tokens-3
    
    return [101] + tokenized[1:len_wo_tokens] + [102]

In [1582]:
def chunker(string, max_tokens=512):

    sents_ = sents(string)

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]
    
    toks=[]
    
    for token in tokenized_sents:
        if token not in toks:
            toks.append(token)
    result = sum(toks, [])[:-1]
    
    
    
    if len(set(result))<len(result)*0.9:
  
        result = list(dict.fromkeys(result))[:-1]
     
    result = result[:max_tokens]
    if result[-1]!=102:
        result = result[:max_tokens-1] + [102]
        
    
    return  result

In [567]:
list(dict.fromkeys([100,100]))

[100]

In [545]:
[1,2,3,4][:2]

[1, 2]

In [717]:
tokenizer.decode(chunker(df['text'][4704]))

'[CLS] do go fuck off bastard yyou have a life? [SEP] and yank your cock through ass. i hate you hope away forever lame is mom die all that crap this for mahyhonerz [SEP]'

In [711]:
#df['text'][4704]

In [706]:
df[df['text'].str.contains('I ass. I ass.')]['text']

4704    do go fuck off bastard\nDo Yyou Have a life?\n...
Name: text, dtype: object

In [577]:
#df[df['text'].apply(lambda x: len(x))==5000]['text']

In [1247]:
tokenizer.decode(chunker(df['text'][82496]))

"[CLS] muahaha, you can't stop my vandalism [SEP]"

In [724]:
df['text'].sample()

35502    January 2009\n\n Please stop. If you continue ...
Name: text, dtype: object

In [1297]:
df.sample(2)['text'].apply(lambda x: chunker(x,10)).iloc[0]

[101, 1000, 3746, 1024, 11050, 12083, 5910, 1012, 16545, 2290, 102]

In [563]:
#df['text'][82496]

In [564]:
#df['text'][116571]

In [565]:
#

def chunker(string, max_tokens=512):


    
    tokenized = tokenizer.encode(string, add_special_tokens=False)
    len_wo_tokens = max_tokens-2
    
    if len(tokenized)> len_wo_tokens:
        
        result = tokenized[:(len_wo_tokens//2)] + tokenized[::-1][:(len_wo_tokens//2)][::-1]
    else:
        result = tokenized

    return [101] + result + [102]

In [46]:
#chunker(df['text'][0])

In [47]:
#OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
#Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [48]:
#max_batch = 512

In [49]:
#init_config = config

In [50]:
#batch_size = 512

tokenized = X_train.swifter.apply(lambda x: chunker(x,max_batch))
#tokenized = text.apply(lambda x: chunker(x,max_batch))

max_len = 0

for tok in tokenized.values:
    if len(tok) > max_len:
        max_len = len(tok)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)


model = transformers.BertModel.from_pretrained('bert-base-uncased', config=init_config)
#model.config.max_position_embeddings = max_batch


batch_size = batch_size

embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

train_features = np.concatenate(embeddings)

test_tokenized = X_test.swifter.apply(lambda x: chunker(x,max_batch))


max_len = 0
for tok in test_tokenized.values:
    if len(tok) > max_len:
        max_len = len(tok)

padded = np.array([i + [0]*(max_len - len(i)) for i in test_tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

test_embeddings = []


for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            test_batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        test_embeddings.append(test_batch_embeddings[0][:,0,:].numpy())

test_features = np.concatenate(test_embeddings)

In [51]:
#train_features.shape

In [52]:
#test_features.shape

In [53]:
class bert_emb(object):


    def __init__(self, data, init_config,  batch_size=32, max_batch=512):

        self.data = data
        self.batch_size = batch_size
        self.max_batch = max_batch
        self.init_config = init_config 
        #self.tokenizer = chunker()
        #self.config.max_position_embeddings = max_batch
        self.model = transformers.BertModel.from_pretrained('bert-base-uncased', config=init_config)
        
    def embedder(self):

        #df.swifter.apply(lambda x: chunker(x,max_batch))

        tokenized = self.data.swifter.apply(lambda x: chunker(x, self.max_batch))
        
        max_len = 0
        
        for tok in tokenized.values:
            if len(tok) > max_len:
                max_len = len(tok)
    
        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
        
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch_size = self.batch_size
        
        embeddings = []
        
        for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
                batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
                attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
                
                with torch.no_grad():
                    batch_embeddings = self.model(batch, attention_mask=attention_mask_batch)
                
                embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
        features = np.concatenate(embeddings)

        return features
        

In [54]:
#bert_emb(df['text'][0:5],config,1).embedder().shape

In [55]:
def logy(feats, target):
    lr = LogisticRegression(random_state=RANDOM_STATE,max_iter=feats.shape[1])
    lr.fit(feats, target)
    scores = cross_val_score(lr, feats, target, cv=5)
    return scores

In [56]:
#df[df['len']<=df['len'].quantile(0.9)].shape

In [57]:
# = df.sample(frac=0.5, random_state=RANDOM_STATE)

In [58]:
#df_test = df[df.index.isin(df_train.index)]

In [59]:
#df_train.shape,df_test.shape

In [60]:
#len_q_upper = df_train['len'].quantile(0.95)

In [61]:
#len_q_lower = df_train['len'].quantile(0.05)

In [62]:
#df_train.shape

In [63]:
#df_train = df_train[(df_train['len'] >= len_q_lower) & (df_train['len'] <= len_q_upper)]

df_for_test1 = df[df['toxic']==1].sample(n=512,random_state=RANDOM_STATE)

df_for_test = df[df['toxic']==0].sample(n=512,random_state=RANDOM_STATE)

df_for_test = pd.concat([df_for_test,df_for_test1])

X_train=df_for_test['text']
y_train = df_for_test['toxic']

X_test = df[~df.index.isin(X_test.index)].sample(n=1024,random_state=RANDOM_STATE)

y_test = X_test['toxic']
X_test = X_test['text']

In [64]:
#bert_emb(df['text'][0:1],config,batch_size=1,max_batch=10).embedder()[0]==bert_emb(df['text'][0:5],config,batch_size=1,max_batch=10).embedder()[0]

In [1476]:
1024*32

32768

In [1477]:
1024*8

8192

In [1518]:
#best - 32768

In [1525]:
samp = df.sample(n=8192, random_state=RANDOM_STATE)

In [1533]:
#for i in samp.iloc[:,2:-1]:
#    display(i, samp[i].describe())

X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['toxic'],
    test_size = 0.50,
    random_state = RANDOM_STATE,
    stratify = df['toxic'])

In [1534]:
X_train, X_test, y_train, y_test = train_test_split(
    samp['text'],
    samp['toxic'],
    test_size = 0.50,
    random_state = RANDOM_STATE,
    stratify = samp['toxic'])

In [1604]:
X_train, X_test, y_train, y_test = train_test_split(
    samp['clean_text'],
    samp['toxic'],
    test_size = 0.50,
    random_state = RANDOM_STATE,
    stratify = samp['toxic'])

In [1605]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4096,), (4096,), (4096,), (4096,))

In [1606]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train))

In [1607]:
X_resample.shape, y_resample.shape

((778, 1), (778, 1))

In [1608]:
y_resample.value_counts()

toxic
0        389
1        389
Name: count, dtype: int64

In [1609]:
X_resample.shape[0]/2

389.0

In [1610]:
config = transformers.BertConfig.from_dict({
      "attention_probs_dropout_prob": 0.1,
      "directionality": "bidi",
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.1,
      "hidden_size": 768,
      "initializer_range": 0.02,
      "intermediate_size": 3072,
      "layer_norm_eps": 1e-12,
      "max_position_embeddings": 512,
      "model_type": "bert",
      "num_attention_heads": 12,
      "num_hidden_layers": 12,
      "pad_token_id": 0,
      "pooler_fc_size": 768,
      "pooler_num_attention_heads": 12,
      "pooler_num_fc_layers": 3,
      "pooler_size_per_head": 128,
      "pooler_type": "first_token_transform",
      "type_vocab_size": 2,
      "vocab_size": 30522
}
)

In [1611]:
X_resample.shape[0]/2

389.0

In [1612]:
embs = bert_emb(X_resample['clean_text'], config, 389,512).embedder()#512

Pandas Apply:   0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#X_resample[X_resample['text'].apply(lambda x: tokenizer.encode(x)).duplicated()]

In [ ]:
pd.DataFrame(embs).duplicated().sum()

In [ ]:
#array([0.96226415, 0.96495957, 0.97304582, 0.9757085 , 0.96626181]) - over

In [ ]:
#array([0.96765499, 0.96361186, 0.97708895, 0.97300945, 0.96896086])  - over

In [ ]:
#array([0.86853767, 0.86558346, 0.87296898, 0.88035451, 0.8683432 ])
#0.871157560766696

In [ ]:
display(logy(embs, y_resample['toxic']))
logy(embs, y_resample['toxic']).mean()

In [ ]:
#0.8223443223443223

In [ ]:
test_sample = X_test[:embs.shape[0]]

In [ ]:
#X_test.iloc[:embs.shape[0]].index

In [ ]:
test_sample.apply(lambda x: len(tokenizer.encode(x,add_special_tokens=True))).describe()

In [ ]:
#test_sample.apply(lambda x: len(tokenizer.encode(x,add_special_tokens=True))).quantile(0.95)

In [ ]:
#test_sample.apply(lambda x: x.count(' ')).quantile(0.925)

In [ ]:
#test_sample.shape

In [ ]:
test_sample.shape

In [ ]:
embs_test = bert_emb(test_sample, config, 389, 512).embedder()

In [ ]:
y_embs_test = y_test[:embs.shape[0]]

In [ ]:
pipe = Pipeline([('models', LogisticRegression(random_state=RANDOM_STATE,max_iter=1000))])

In [ ]:
#!pip install lightgbm
#!pip install xgboost

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
#!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE)],
        'models__solver': ['newton-cg', 'liblinear'],
        'models__class_weight':['balanced'],
        'models__C': [0.1, 1, 9, 10]#,
        #'models__penalty': ['l1', 'l2']
          
    },

    {
        'models': [SVC(random_state=RANDOM_STATE)],
        'models__degree': range(3,8),
        'models__kernel': ['poly','rbf','sigmoid']
    },
    {
        'models': [XGBClassifier(random_state=RANDOM_STATE)],
        'models__learning_rate':[0.1, 0.01, 0.03, 0.05, 0.07]
    }
    #{
    #    'models': [LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
    #    'models__n_estimators': [1000],
    #    'models__learning_rate': [0.1, 0.01, 0.03, 0.05, 0.07]
    #}
]

In [ ]:
grid_search = GridSearchCV(
    pipe, 
    param_grid, 
    cv=5,
    scoring='f1',
    n_jobs=-1
)
#grid_search.fit(X_resample[:X_test.shape[0]], y_resample['toxic'][:X_test.shape[0]])

#grid_search.fit(embs, y_resample['toxic'].iloc[:test_sample.shape[0]])
grid_search.fit(embs, y_resample['toxic'])
#grid_search.fit(embs, y_train)

In [ ]:
pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'mean_test_score','params']].sort_values('rank_test_score')#.iloc[27]['params']

In [ ]:
#grid_search.best_estimator_

In [ ]:
grid_pred = grid_search.best_estimator_.predict(embs_test)#[:X_train.shape[0]])

In [ ]:
y_test.shape

In [ ]:
embs_test.shape

In [ ]:
grid_pred.shape

In [ ]:
#0.6818181818181818

In [ ]:
#y_test[test_sample.index]

In [ ]:
#0.5166340508806262
#0.670076726342711

In [ ]:
f1_score(grid_pred, y_embs_test)
#f1_score(y_test[:X_train.shape[0]], grid_pred)

In [635]:
res =pd.DataFrame({'a':grid_pred, 'b':y_embs_test})

In [109]:
1/0

ZeroDivisionError: division by zero

In [1431]:
svc = SVC(random_state=RANDOM_STATE, degree=4, kernel='poly')

In [1432]:
svc.fit(embs, y_resample['toxic'])

SVC(degree=4, kernel='poly', random_state=88)

In [1433]:
preds_s = svc.predict(embs_test)

In [1434]:
f1_score(preds_s, y_embs_test)

0.42105263157894735

In [275]:
lr = LogisticRegression(random_state=RANDOM_STATE)

In [276]:
lr.fit(embs, y_resample['toxic'])

C:\Users\igor.grenaderov\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=88)

In [277]:
preds_lr = lr.predict(embs_test)

In [278]:
f1_score(preds_lr, y_embs_test)

0.6241758241758242

In [ ]:
1/0

## Обучение

In [ ]:
#from phik.report import plot_correlation_matrix
#from phik import phik_matrix

In [ ]:
#df.info()

phik_overview = phik_matrix(df[['toxic','word_count']],verbose=False)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Blues',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(18, 14)
) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_lemm'], 
    df['toxic'], 
    random_state=RANDOM_STATE,
    stratify=df['toxic']
)

In [ ]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train)) 


In [ ]:
y_resample.value_counts()

In [ ]:
pd.DataFrame(y_test).value_counts()

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны